# create global map projections from model predictions

create .nc file with each layer being the data for a feature at every lat/lon for a particular year, month, and depth

for the purposes of today, I'll make four .nc files:
* 2009-Jan-surface
* 2009-Apr-surface
* 2009-July-surface
* 2009-Oct-surface

Resolution to use: 

out contemporary data is at 9km resolution (except chl is at 4km); our historical data is 0.5 or 1 degree resolution. I'm unsure about whether to use 9km res - in which case the 1-degree res value will be repeated for several points, but the variability in the 9km contemporary data will be captured - or whether to downsample the 9km data to be lower resolution. 

For now, I'm going to use 9km resolution. Not sure if this is really best practices but let's see how results look. 

TODO: downsample chl to 9km at some point.

## create netcdf

In [3]:
#lat/lon at 9km resolution - grab it from sst
import netCDF4 as nc
import datetime as dt
import numpy as np
import pandas as pd


In [24]:
projections.close()

In [16]:
projections = nc.Dataset("envforprojections_2009.nc", "r+", format="NETCDF4")
projections

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    missing_value: -9999
    dimensions(sizes): 
    variables(dimensions): 
    groups: contemporary, annual, monthly

In [17]:
contemp = projections['contemporary']
monthly = projections['monthly']
annual = projections['annual']

In [97]:
#create 3 groups: contemporary, annual historical, monthly historical 
contemp = projections.createGroup("contemporary")
annual = projections.createGroup("annual")
monthly = projections.createGroup("monthly")

#add dimensions - 
#contemporary dims should be lat, lon, and time which will be unlimited (but for now 4 layers with quarterly months (Jan 2009, Apr 2009, July 2009, Oct 2009))
lat_contemp = contemp.createDimension("lat", 2160)
lon_contemp = contemp.createDimension("lon", 4320)
time_contemp = contemp.createDimension("time", 0)
#create lat and lon and time variables as 64-bit floats and ints
latitudes_contemp = contemp.createVariable("lat","f8",("lat",))
longitudes_contemp = contemp.createVariable("lon","f8",("lon",))
times_contemp = contemp.createVariable("time", "f8", ("time",))

#monthly historical dims lat, lon, and time with 12 levels
lat_monthly = monthly.createDimension("lat", 2160)
lon_monthly = monthly.createDimension("lon", 4320)
time_monthly = monthly.createDimension("time", 12)
#create lat, lon, time variables
latitudes_monthly = monthly.createVariable("lat", "f8", ("lat",))
longitudes_monthly = monthly.createVariable("lon", "f8", ("lon",))
times_monthly = monthly.createVariable("time", "f8", ("time",))

#annual historical dims lat, lon only
lat_annual = annual.createDimension("lat", 2160)
lon_annual = annual.createDimension("lon", 4320)
#lat, lon variables
latitudes_annual = annual.createVariable("lat", "f8", ("lat",))
longitudes_annual = annual.createVariable("lon", "f8", ("lon",))

#assign lat/lon values at 9km res to the lat/lon variables 
lats = np.array(sst['lat'][:])
lons = np.array(sst['lon'][:])
contemp['lat'][:] = lats
contemp['lon'][:] = lons
monthly['lat'][:] = lats
monthly['lon'][:] = lons
annual['lat'][:] = lats
annual['lon'][:] = lons


'\nprojections = nc.Dataset("envforprojections_2009.nc", "w", format="NETCDF4")\n#create 3 groups: contemporary, annual historical, monthly historical \ncontemp = projections.createGroup("contemporary")\nannual = projections.createGroup("annual")\nmonthly = projections.createGroup("monthly")\n\n#add dimensions - \n#contemporary dims should be lat, lon, and time which will be unlimited (but for now 4 layers with quarterly months (Jan 2009, Apr 2009, July 2009, Oct 2009))\nlat_contemp = contemp.createDimension("lat", 2160)\nlon_contemp = contemp.createDimension("lon", 4320)\ntime_contemp = contemp.createDimension("time", 0)\n#create lat and lon and time variables as 64-bit floats and ints\nlatitudes_contemp = contemp.createVariable("lat","f8",("lat",))\nlongitudes_contemp = contemp.createVariable("lon","f8",("lon",))\ntimes_contemp = contemp.createVariable("time", "f8", ("time",))\n\n#monthly historical dims lat, lon, and time with 12 levels\nlat_monthly = monthly.createDimension("lat", 

In [88]:
#assign time values to contemporary and monthly
contemp['time'][:] = [14275.,14364.,14456.,14548.]
monthly['time'][:] = [1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.]

## insert contemporary data 

In [23]:
def insert_contemp(var, var_target, source_nc, target_nc):
    #create variable and attributes
    if var_target in target_nc.variables:
        var_target = target_nc[var_target]
    else:
        var_target = target_nc.createVariable(var_target,"f8",("lat","lon","time"))

    #set attributes as same as original
    for attr in source_nc[var].ncattrs():
        var_target.setncattr(attr, str(getattr(source_nc[var], attr)))
    #insert into projections
    if source_nc[var].shape == (72,2160,4320):
        for day_ix, day in enumerate(target_nc['time'][:]):
            subset = np.array(source_nc[var][int(np.argwhere(source_nc['time'][:]==target_nc['time'][day_ix])),:,:])
            if np.ma.is_masked(subset):
                print("filling masked")
                subset = subset.filled(fill_value=np.nan) 
            #insert into projections 
            var_target[:,:,day_ix] = subset
    else:
        print('shape not as expected')


In [183]:
sst = nc.Dataset("../satellite_models/get_env_data/satellite_data/sst_compiled.nc")
insert_contemp(var='sst', var_target='sst_satellite', source_nc=sst, target_nc=contemp)

#### downsample chl to 9km res and save as new .nc

In [7]:
chl = nc.Dataset("../satellite_models/get_env_data/satellite_data/chla_monthly_compiled.nc")
chl

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    institution: NASA Goddard Space Flight Center, Ocean Ecology Laboratory, Ocean Biology Processing Group
    references: http://oceandata.sci.gsfc.nasa.gov
    source: MODIS
    history: Formatted on 2017-11-27; smigen par
    title: MODIS Level-3 Standard Mapped Image
    dimensions(sizes): lat(4320), lon(8640), time(72), nv(2)
    variables(dimensions): float64 lat(lat), float64 lat_bnds(lat,nv), float64 lon(lon), float64 lon_bnds(lon,nv), float64 time(time), float64 climatology_bounds(time,nv), float64 chlor_a(time,lat,lon)
    groups: 

In [54]:
chl_9km = nc.Dataset("../satellite_models/get_env_data/satellite_data/chla_monthly_compiled_9km.nc", "w", format="NETCDF4")

#add dimensions - 
lat_chl = chl_9km.createDimension("lat", 2160)
lon_chl = chl_9km.createDimension("lon", 4320)
time_chl = chl_9km.createDimension("time", 72)

#create lat and lon and time variables as 64-bit floats and ints
latitudes_chl = chl_9km.createVariable("lat","f8",("lat",))
longitudes_chl = chl_9km.createVariable("lon","f8",("lon",))
times_chl = chl_9km.createVariable("time", "f8", ("time",))

#add lat/lon/time values to nc
chl_9km['lat'][:] = contemp['lat'][:]
chl_9km['lon'][:] = contemp['lon'][:]
chl_9km['time'][:] = chl['time'][:]

#add chl variable
var_chl = chl_9km.createVariable("chlor_a","f8",("time","lat","lon"))

OSError: Permission denied

In [38]:
from mpl_toolkits import basemap
lat_source = chl['lat'][:]
lon_source = chl['lon'][:]
lons_sub, lats_sub = np.meshgrid(lon_source[::2], lat_source[::2])

for day_ix, day in enumerate(chl_9km['time'][:]):
    print("processing ", day_ix)
    chl_source = chl['chlor_a'][day_ix,:,:]
    chl_coarse = basemap.interp(chl_source, lon_source, lat_source, lons_sub, lats_sub, order=1)
    #insert into projections 
    chl_9km['chlor_a'][day_ix,:,:] = chl_coarse

chl_9km['chlor_a']

processing  0
processing  1
processing  2
processing  3
processing  4
processing  5
processing  6
processing  7
processing  8
processing  9
processing  10
processing  11
processing  12
processing  13
processing  14
processing  15
processing  16
processing  17
processing  18
processing  19
processing  20
processing  21
processing  22
processing  23
processing  24
processing  25
processing  26
processing  27
processing  28
processing  29
processing  30
processing  31
processing  32
processing  33
processing  34
processing  35
processing  36
processing  37
processing  38
processing  39
processing  40
processing  41
processing  42
processing  43
processing  44
processing  45
processing  46
processing  47
processing  48
processing  49
processing  50
processing  51
processing  52
processing  53
processing  54
processing  55
processing  56
processing  57
processing  58
processing  59
processing  60
processing  61
processing  62
processing  63
processing  64
processing  65
processing  66
proce

<class 'netCDF4._netCDF4.Variable'>
float64 chlor_a(time, lat, lon)
unlimited dimensions: 
current shape = (72, 2160, 4320)
filling on, default _FillValue of 9.969209968386869e+36 used

In [57]:
for attr in chl['chlor_a'].ncattrs():
    chl_9km.setncattr(attr, str(getattr(chl['chlor_a'], attr)))
chl_9km['chlor_a']

RuntimeError: NetCDF: Not a valid ID

In [61]:
#save
chl_9km.close()


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lat(2160), lon(4320), time(72)
    variables(dimensions): float64 lat(lat), float64 lon(lon), float64 time(time), float64 chlor_a(time,lat,lon)
    groups: 

In [42]:
#reopen to check
chl_9km = nc.Dataset("../satellite_models/get_env_data/satellite_data/chla_monthly_compiled_9km.nc")
chl_9km['chlor_a']

<class 'netCDF4._netCDF4.Variable'>
float64 chlor_a(time, lat, lon)
unlimited dimensions: 
current shape = (72, 2160, 4320)
filling on, default _FillValue of 9.969209968386869e+36 used

### okay insert chl

In [69]:
insert_contemp(var='chlor_a', var_target='chl_satellite', source_nc=chl_9km, target_nc=contemp)

### insert the rest of contemporary

In [79]:
#read in npp
npp = nc.Dataset('../satellite_models/get_env_data/satellite_data/npp_monthly_compiled.nc')
npp['npp']

<class 'netCDF4._netCDF4.Variable'>
float64 npp(time, lat, lon)
    units: mgC m-2 day-1
    missing_value: -9999
    long_name: Net Primary Productivity
    cell_methods: not specified
unlimited dimensions: time
current shape = (72, 2160, 4320)
filling off

In [82]:
insert_contemp(var='npp', var_target='npp_satellite', source_nc=npp, target_nc=contemp)

In [88]:
#read in par
par = nc.Dataset('../satellite_models/get_env_data/satellite_data/par_monthly_compiled.nc')
par['par']

<class 'netCDF4._netCDF4.Variable'>
float64 par(time, lat, lon)
    units: einstein m^-2 day^-1
    missing_value: -9999
    long_name: Photosynthetically Available Radiation, R. Frouin
    cell_methods: Mean
unlimited dimensions: time
current shape = (72, 2160, 4320)
filling off

In [89]:
insert_contemp(var='par', var_target='par_satellite', source_nc=par, target_nc=contemp)

In [20]:
#poc
poc = nc.Dataset('../satellite_models/get_env_data/satellite_data/poc_monthly_compiled.nc')
subset = poc['poc'][17,:,:]
if np.ma.is_masked(subset):
    print("filling masked")
    subset = subset.filled(fill_value=np.nan)
np.nanmean(subset)


filling masked


72.337080909846534

In [22]:
insert_contemp(var='poc', var_target='poc_satellite', source_nc=poc, target_nc=contemp)

[[-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 ..., 
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]]
False
[[-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 ..., 
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]]
False
[[-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 ..., 
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999. -9999. -9999. ..., -9999. -9999. -9999.]]
False
[[-9999. -9999. -9999. ..., -9999. -9999. -9999.]
 [-9999.

In [6]:
for day in range(0,4):
    subset = contemp['poc_satellite'][:,:,day]
    print(np.nanmean(subset), np.nanstd(subset))

65.8616241657 79.8976673299
66.9748112102 103.823000096
73.8573711838 117.539630357
65.9550013785 101.932406293


In [8]:
#pic
pic = nc.Dataset('../satellite_models/get_env_data/satellite_data/pic_monthly_compiled.nc')
insert_contemp(var='pic', var_target='pic_satellite', source_nc=pic, target_nc=contemp)

## add monthly data

In [7]:
monthly = projections['monthly']
monthly

<class 'netCDF4._netCDF4.Group'>
group /monthly:
    dimensions(sizes): lat(2160), lon(4320), time(12)
    variables(dimensions): float64 lat(lat), float64 lon(lon), uint64 time(time), float64 chla_monthly_historical(lat,lon,time), float64 cloudfraction_monthly_historical(lat,lon,time), float64 daylength_monthly_historical(lat,lon,time), float64 dustflux_monthly_historical(lat,lon,time), float64 solarinsolation_monthly_historical(lat,lon,time), float64 pycnoclinedepth_monthly_historical(lat,lon,time), float64 thermoclinedepth_monthly_historical(lat,lon,time), float64 nitrate_monthly_historical(lat,lon,time), float64 npratio_monthly_historical(lat,lon,time), float64 oxygendissolved_monthly_historical(lat,lon,time), float64 oxygensaturation_monthly_historical(lat,lon,time), float64 oxygenutilization_monthly_historical(lat,lon,time), float64 phosphate_monthly_historical(lat,lon,time), float64 salinity_monthly_historical(lat,lon,time), float64 silicate_monthly_historical(lat,lon,time), flo

In [8]:
'''
## monthly files to insert
[
 #'chloMomeanNASA.nc',
 #'cloudfracMomeanNASA.nc',
  #'daylengthMomeanEarthtools.nc',
 #'dustMomeanJickells.nc',
 #'insolationMomeanNASA.nc',
 #'mixedlayerdensityMomeanMontegut.nc',
 #'mixedlayertempMomeanMontegut.nc',
 #'nitrateMomeanWOA.nc',
 #'npratioMomeanWOA.nc',
 #'oxygendissolvedMomeanWOA.nc',
 #'oxygensaturationMomeanWOA.nc',
 #'oxygenutilizationMomeanWOA.nc',
 #'phosphateMomeanWOA.nc',
 #'salinityMomeanWOA.nc',
 #'silicateMomeanWOA.nc',
 #'temperatureMomeanWOA.nc',
]
 
'''

monthly_info = {
    'chloMomeanNASA.nc' : {
        'var': 'Chlorophyll_Concentration',
        'var_target': 'chla_monthly_historical'
    },
    'cloudfracMomeanNASA.nc' : {
        'var': 'cloud_fraction',
        'var_target': 'cloudfraction_monthly_historical'
    },
    'daylengthMomeanEarthtools.nc' : {
        'var': 'Day_Length_on_15th_Day_of_Month',
        'var_target': 'daylength_monthly_historical'
    },
    'dustMomeanJickells.nc' : {
        'var': 'Dust_Deposition',
        'var_target': 'dustflux_monthly_historical'
    },
    'insolationMomeanNASA.nc' : {
        'var': 'solar_insolation',
        'var_target': 'solarinsolation_monthly_historical'
    },
    'mixedlayerdensityMomeanMontegut.nc' : {
        'var': 'Mixed_Layer_Depth',
        'var_target': 'pycnoclinedepth_monthly_historical'
    },
    'mixedlayertempMomeanMontegut.nc' : {
        'var': 'Mixed_Layer_Depth',
        'var_target': 'thermoclinedepth_monthly_historical'
    },
    'nitrateMomeanWOA.nc' : {
        'var': 'nitrate',
        'var_target': 'nitrate_monthly_historical'
    },
    'npratioMomeanWOA.nc' : {
        'var': 'Nitrate_Phosphate_Ratio',
        'var_target': 'npratio_monthly_historical'
    },
    'oxygendissolvedMomeanWOA.nc' : {
        'var': 'oxygendissolved',
        'var_target': 'oxygendissolved_monthly_historical'
    },
    'oxygensaturationMomeanWOA.nc' : {
        'var': 'oxygensaturation',
        'var_target': 'oxygensaturation_monthly_historical'
    },
    'oxygenutilizationMomeanWOA.nc' : {
        'var': 'oxygenutilization',
        'var_target': 'oxygenutilization_monthly_historical'
    },
    'phosphateMomeanWOA.nc' : {
        'var': 'phosphate',
        'var_target': 'phosphate_monthly_historical'
    },
    'salinityMomeanWOA.nc' : {
        'var': 'salinity',
        'var_target': 'salinity_monthly_historical'
    },
    'silicateMomeanWOA.nc' : {
        'var': 'silicate',
        'var_target': 'silicate_monthly_historical'
    },
    'temperatureMomeanWOA.nc' : {
        'var': 'temperature',
        'var_target': 'oceantemp_monthly_historical'
    }
}


In [9]:
def insert_monthly(var, var_target, source_nc, target_nc):
    from mpl_toolkits import basemap
    #create variable and attributes
    if var_target in target_nc.variables:
        var_target = target_nc[var_target]
    else:
        var_target = target_nc.createVariable(var_target,"f8",("lat","lon","time"))

    #set attributes as same as original
    for attr in source_nc[var].ncattrs():
        var_target.setncattr(attr, str(getattr(source_nc[var], attr)))
    #insert into projections

    print("processing ", var)
    #change resolution
    lats_source = source_nc['lat'][:]
    lons_source = source_nc['lon'][:]
    lats_fine = target_nc['lat'][:]
    lons_fine = target_nc['lon'][:]
    lons_sub, lats_sub = np.meshgrid(lons_fine, lats_fine)

    if source_nc[var].shape in [(360,720,1,12), (360, 720, 14, 12)]:
        for mo_ix, month in enumerate(target_nc['time'][:]):
            var_source = source_nc[var][:,:,0,mo_ix]
            var_fine = basemap.interp(var_source, lons_source, lats_source, lons_sub, lats_sub, order=1)
            if np.ma.is_masked(var_fine):
                var_fine = var_fine.filled(fill_value=np.nan)
            #insert into projections 
            var_target[:,:,mo_ix] = var_fine
            
    elif source_nc[var].shape in [(12, 37, 180, 360), (12, 57, 180, 360)]:
        #need to use len(vert)-2 layer; 36 or 56
        vert_layer = len(source_nc['vert'][:])-2
        print('using vertical layer ', vert_layer, ' for ', var)
        for mo_ix, month in enumerate(target_nc['time'][:]):
            var_source = source_nc[var][mo_ix,vert_layer,:,:]
            var_fine = basemap.interp(var_source, lons_source, lats_source, lons_sub, lats_sub, order=1)
            if np.ma.is_masked(var_fine):
                var_fine = var_fine.filled(fill_value=np.nan)
            #insert into projections 
            var_target[:,:,mo_ix] = var_fine
            
    else:
        print('shape not as expected')

#insert_monthly(var="Chlorophyll_Concentration",var_target="chl_monthly_historical", source_nc=chlmo, target_nc=monthly)

In [10]:
for key in monthly_info.keys():
    nc_data = nc.Dataset('../satellite_models/get_env_data/historical_data/'+str(key))
    insert_monthly(var=monthly_info[key]['var'],var_target=monthly_info[key]['var_target'], source_nc=nc_data, target_nc=monthly)

processing  nitrate
using vertical layer  35  for  nitrate
processing  cloud_fraction
processing  Mixed_Layer_Depth
processing  Chlorophyll_Concentration
processing  silicate
using vertical layer  35  for  silicate
processing  oxygensaturation
using vertical layer  55  for  oxygensaturation
processing  phosphate
using vertical layer  35  for  phosphate
processing  solar_insolation
processing  oxygenutilization
using vertical layer  55  for  oxygenutilization
processing  Day_Length_on_15th_Day_of_Month
processing  oxygendissolved
using vertical layer  55  for  oxygendissolved
processing  salinity
using vertical layer  55  for  salinity
processing  Dust_Deposition
processing  Nitrate_Phosphate_Ratio
processing  temperature
using vertical layer  55  for  temperature
processing  Mixed_Layer_Depth


## add annual historical to .nc

In [11]:
'''
 ##annual files to insert
 'calciteAnmeanBiooracle.nc',
 'chlorAnmeanBiooracle.nc',
 'chlorAnrangeBiooracle.nc',
 'cldAnmeanBiooracle.nc',
 'cloudfracStdevNASA.nc',
 'daAnmeanBiooracle.nc',
 'dustAnmeanJickells.nc',
 'dustStdevJickells.nc',
 'insolationAnmeanBiooracle.nc',
 'insolationStdevNASA.nc',
 'landdistAnmeanReady.nc',
 'mixedlayerdensityStdevMontegut.nc',
  'mixedlayertempStdevMontegut.nc',
'nitrateAnmeanWOA.nc',
 'oceandepthAnmeanNASA.nc',
 'oxygendissolvedAnmeanWOA.nc',
  'oxygensaturationAnmeanWOA.nc',
 'oxygenutilizationAnmeanWOA.nc',
 'parAnmeanBiooracle.nc', 
  'phAnmeanBiooracle.nc',
 'phosphateAnmeanWOA.nc',
  'salinityAnmeanWOA.nc',
'silicateAnmeanWOA.nc',
 'sstAnmeanBiooracle.nc',
 'watertempAnmeanWOA.nc',
'''

annual_info = {
    'calciteAnmeanBiooracle.nc' : {
        'var': 'calcite',
        'var_target': 'calcite_annual_historical'
    },
    'chlorAnmeanBiooracle.nc':{
        'var':'chlor',
        'var_target': 'chla_annual_historical'
    },
    'chlorAnrangeBiooracle.nc': {
        'var': 'chlorrange',
        'var_target': 'chla_annualrange_historical'
    },
    'cldAnmeanBiooracle.nc': {
        'var': 'cld',
        'var_target': 'cloudfraction_annual_historical'
    },
    'cloudfracStdevNASA.nc': {
        'var':'AnnualStdev_Cloud_Fraction',
        'var_target': 'cloudfraction_annualstdev_historical'
    },
    'daAnmeanBiooracle.nc' : {
        'var': 'Diffuse_attenuation_coefficient',
        'var_target': 'diffuseattenuation_annual_historical'
    },
    'dustAnmeanJickells.nc': {
        'var':'Dust_Deposition',
        'var_target': 'dustflux_annual_historical'
    },
    'dustStdevJickells.nc': {
        'var':'AnnualStdev_Dust_Deposition',
        'var_target': 'dustflux_annualstdev_historical'
    },
    'insolationAnmeanBiooracle.nc': {
        'var':'Solar_Insolation',
        'var_target':'solarinsolation_annual_historical'
    },
    'insolationStdevNASA.nc': {
        'var': 'AnnualStdev_Solar_Insolation',
        'var_target': 'solarinsolation_annualstdev_historical'
    },
    'landdistAnmeanReady.nc' : {
        'var': 'LandDist',
        'var_target': 'distfromland_annual_historical'
    },
    'mixedlayerdensityStdevMontegut.nc' : {
        'var': 'AnnualStdev_Mixed_Layer_Depth_02',
        'var_target': 'pycnoclinedepth_annualstdev_historical'
    },
    'mixedlayertempStdevMontegut.nc' : {
        'var': 'AnnualStdev_Mixed_Layer_Depth_11',
        'var_target': 'thermoclinedepth_annualstdev_historical'
    },
    'nitrateAnmeanWOA.nc' : {
        'var': 'nitrate',
        'var_target': 'nitrate_annual_historical'
    },
    'oceandepthAnmeanNASA.nc' : {
        'var': 'DepthMean',
        'var_target': 'oceandepth_historical'
    },
    'oxygendissolvedAnmeanWOA.nc' : {
        'var': 'oxygendissolved',
        'var_target': 'oxygendissolved_annual_historical'
    },
    'oxygensaturationAnmeanWOA.nc' : {
        'var': 'oxygensaturation',
        'var_target': 'oxygensaturation_annual_historical'
    },
    'oxygenutilizationAnmeanWOA.nc' : {
        'var': 'oxygenutilization',
        'var_target': 'oxygenutilization_annual_historical'
    },
    'parAnmeanBiooracle.nc' : {
        'var': 'par',
        'var_target': 'par_annual_historical'
    },
    'phAnmeanBiooracle.nc' : {
        'var': 'pH',
        'var_target': 'ph_annual_historical'
    },
    'phosphateAnmeanWOA.nc' : {
        'var': 'phosphate',
        'var_target': 'phosphate_annual_historical'
    },
    'salinityAnmeanWOA.nc' : {
        'var': 'salinity',
        'var_target': 'salinity_annual_historical'
    },
    'silicateAnmeanWOA.nc' : {
        'var': 'silicate',
        'var_target': 'silicate_annual_historical'
    },
    'sstAnmeanBiooracle.nc' : {
        'var': 'Sea_surface_temperature_mean',
        'var_target': 'sst_annual_historical'
    },
    'watertempAnmeanWOA.nc' : {
        'var': 'watertemp',
        'var_target': 'oceantemp_annual_historical'
    }
    
}

In [13]:
def insert_annual(var, var_target, source_nc, target_nc):
    from mpl_toolkits import basemap
    #create variable and attributes
    if var_target in target_nc.variables:
        var_target = target_nc[var_target]
    else:
        var_target = target_nc.createVariable(var_target,"f8",("lat","lon"))

    #set attributes as same as original
    for attr in source_nc[var].ncattrs():
        var_target.setncattr(attr, str(getattr(source_nc[var], attr)))
    #insert into projections

    print("processing ", var)
    #change resolution
    lats_source = source_nc['lat'][:]
    lons_source = source_nc['lon'][:]
    lats_fine = target_nc['lat'][:]
    lons_fine = target_nc['lon'][:]
    lons_sub, lats_sub = np.meshgrid(lons_fine, lats_fine)

    if source_nc[var].shape in [(1, 2160, 4320)]:
        var_source = source_nc[var][0,:,:]
        var_fine = var_source
          
    elif source_nc[var].shape in [(1, 102, 180, 360)]:
        print('using level 100 for ', var)
        #gotta use len(vert)-2 level to get surface data (centered around 5m)
        var_source = source_nc[var][0,100,:,:]
        var_fine = basemap.interp(var_source, lons_source, lats_source, lons_sub, lats_sub, order=1)
        
    elif source_nc[var].shape in [(360, 720)]:
        var_source = source_nc[var][:,:]
        var_fine = basemap.interp(var_source, lons_source, lats_source, lons_sub, lats_sub, order=1)
        
    else:
        print('shape not as expected')
        
    if np.ma.is_masked(var_fine):
        print("filling masked")
        var_fine = var_fine.filled(fill_value=np.nan)
        #insert into projections 
    var_target[:,:] = var_fine

#insert_annual(var="chlor",var_target="chla_annual_historical", source_nc=chlan, target_nc=annual)

In [14]:
for key in annual_info.keys():
    nc_data = nc.Dataset('../satellite_models/get_env_data/historical_data/'+str(key))
    insert_annual(var=annual_info[key]['var'],var_target=annual_info[key]['var_target'], source_nc=nc_data, target_nc=annual)

processing  oxygensaturation
using level 100 for  oxygensaturation
processing  watertemp
using level 100 for  watertemp
processing  Sea_surface_temperature_mean
filling masked
processing  DepthMean
filling masked
processing  chlor
processing  silicate
using level 100 for  silicate
processing  AnnualStdev_Solar_Insolation
processing  AnnualStdev_Mixed_Layer_Depth_02
filling masked
processing  calcite
processing  AnnualStdev_Mixed_Layer_Depth_11
filling masked
processing  LandDist
filling masked
processing  AnnualStdev_Cloud_Fraction
filling masked
processing  Dust_Deposition
processing  AnnualStdev_Dust_Deposition
processing  par
processing  cld
processing  phosphate
using level 100 for  phosphate
processing  salinity
using level 100 for  salinity
processing  oxygenutilization
using level 100 for  oxygenutilization
processing  chlorrange
processing  Diffuse_attenuation_coefficient
filling masked
processing  oxygendissolved
using level 100 for  oxygendissolved
processing  nitrate
using l

## close projections to save

In [15]:
projections.close()

## write functions to make lat, lon matrix of predictions and confidence scores 

extract feat 2160x4320 matrix for each feat, multiply by its weight in model; add together to get matrix of scores
apply sigmoid to matrix to get prediction probs
write predictions and confidences as layers in prediction .nc file

1) fn to get 67 2160x4320 feature matrices (for each feat) 

2) fn to multiply vector of weights (for a gene model; a column of balanced) by the matrix for feat matrices, and sum the 67 matrices ==> score matrix

3) fn to apply sigmoid fn to the matrix ==> prediction probabilities matrix

4) fn to binarize to presence/absence ==> presence/absence matrix




### just testing some matrix ops

In [339]:
test = np.array([[1,2,3],[20,30,40]])
print(test)
test2 = np.array([[-5,-10,-15],[-1,5,-1]])
print(test2)

[[ 1  2  3]
 [20 30 40]]
[[ -5 -10 -15]
 [ -1   5  -1]]


In [340]:
test * 2

array([[ 2,  4,  6],
       [40, 60, 80]])

In [396]:
test3 = np.array([test, test2])
test3

array([[[  1,   2,   3],
        [ 20,  30,  40]],

       [[ -5, -10, -15],
        [ -1,   5,  -1]]])

In [383]:
print(test3.shape)
print(test3)
wei = [2,3]
weights = np.array([np.broadcast_to(w,shape=(2,3)) for w in wei])
print(weights.shape)
weighted_test = test3*weights
print(weighted_test)

(2, 2, 3)
[[[  1   2   3]
  [ 20  30  40]]

 [[ -5 -10 -15]
  [ -1   5  -1]]]
(2, 2, 3)
[[[  2   4   6]
  [ 40  60  80]]

 [[-15 -30 -45]
  [ -3  15  -3]]]


In [386]:
#sum all the scores of each 2x3 matrix
weighted_test.sum(axis=0)

array([[-13, -26, -39],
       [ 37,  75,  77]])

In [6]:
contemp['sst_satellite'].shape

(2160, 4320, 4)

### get feature weights and reorder in expected order

In [18]:
balanced = pd.read_csv('../satellite_models/BalancedGenes_LogRegCoefficients_L1.csv')
balanced.index = balanced['feature']
balanced.head()

,feature,fig|1123865.4.peg.747|VBISARClu228887_0747|,fig|1123866.3.peg.1001|VBISARClu227658_1001|,fig|1123866.3.peg.1002|VBISARClu227658_1002|,fig|1123866.3.peg.1003|VBISARClu227658_1003|,fig|1123866.3.peg.1004|VBISARClu227658_1004|,fig|1123866.3.peg.1005|VBISARClu227658_1005|,fig|1123866.3.peg.1006|VBISARClu227658_1006|,fig|1123866.3.peg.1007|VBISARClu227658_1007|,fig|1123866.3.peg.1009|VBISARClu227658_1009|,...,scf7180009409900_97374_97712_93,scf7180009409900_97712_98362_94,scf7180009409900_9787_10266_11,scf7180009409900_98373_98777_95,scf7180009409900_98781_99002_96,scf7180009409900_98999_99301_97,scf7180009409900_99298_99666_98,scf7180009409900_99669_99995_99,scf7180009409900_99998_100534_100,sort
feature,,,,,,,,,,,,,,,,,,,,,
latitude,latitude,0.000000,0.000000,0.000000,-0.250108,0.000000,0.153432,0.000000,-0.678068,-0.575633,...,0.000000,0.044585,0.000000,0.000000,0.000000,0.000000,0.000000,0.045311,0.005366,0.005366
longitude,longitude,0.072304,0.582633,0.365639,0.726257,0.820915,0.000000,1.091816,0.721188,0.000000,...,0.423079,0.314069,0.302202,0.534109,0.000000,0.324347,0.000000,0.000000,0.454729,0.454729
depth_sampled,depth_sampled,0.114609,0.000000,0.000000,0.000000,0.000000,0.000000,0.957090,0.000000,-0.129229,...,-0.425184,-0.153393,-0.493410,-0.010246,-0.526781,-0.735575,-0.137729,0.000000,0.000000,0.000000
chl_satellite,chl_satellite,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sst_satellite,sst_satellite,0.000000,1.199042,0.415051,0.397619,0.550943,0.062045,0.277805,1.567771,1.653628,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
general_feats = ['latitude', 'longitude', 'depth_sampled']
satellite_feats = ['chl_satellite', 'sst_satellite', 'par_satellite', 'pic_satellite', 'poc_satellite', 'npp_satellite']
monthly_feats = ['chla_monthly_historical', 'cloudfraction_monthly_historical', 'daylength_monthly_historical', 
                 'dustflux_monthly_historical', 'solarinsolation_monthly_historical', 'pycnoclinedepth_monthly_historical',
                 'thermoclinedepth_monthly_historical', 'nitrate_monthly_historical', 'npratio_monthly_historical',
                 'oxygendissolved_monthly_historical', 'oxygensaturation_monthly_historical', 'oxygenutilization_monthly_historical',
                 'phosphate_monthly_historical', 'salinity_monthly_historical', 'silicate_monthly_historical', 'oceantemp_monthly_historical'
                ]
annual_feats = ['chla_annual_historical', 'chla_annualrange_historical', 'cloudfraction_annual_historical', 'cloudfraction_annualstdev_historical',
                'diffuseattenuation_annual_historical', 'par_annual_historical', 'salinity_annual_historical',
                'thermoclinedepth_annualstdev_historical', 'nitrate_annual_historical', 'solarinsolation_annual_historical',
                'distfromland_annual_historical', 'oxygendissolved_annual_historical', 'sst_annual_historical', 
                'pycnoclinedepth_annualstdev_historical', 'solarinsolation_annualstdev_historical', 'oceandepth_historical', 
                'dustflux_annual_historical', 'oxygensaturation_annual_historical', 'dustflux_annualstdev_historical', 
                'oxygenutilization_annual_historical', 'phosphate_annual_historical', 'silicate_annual_historical', 
                'calcite_annual_historical', 'oceantemp_annual_historical', 'ph_annual_historical'
                ]
year_sampled_feats = ['year_sampled_2009', 'year_sampled_2010', 'year_sampled_2011', 'year_sampled_2012']
month_sampled_feats = ['month_sampled_1', 'month_sampled_2', 'month_sampled_3', 'month_sampled_4', 
                       'month_sampled_5', 'month_sampled_6', 'month_sampled_7', 'month_sampled_8', 
                       'month_sampled_9', 'month_sampled_10', 'month_sampled_11', 'month_sampled_12'
                      ]


feat_order = general_feats+satellite_feats+monthly_feats+annual_feats+year_sampled_feats+month_sampled_feats+['intercept']

In [20]:
balanced = balanced.reindex(feat_order)
balanced.head(15)
print(balanced['feature'])
balanced['fig|1123865.4.peg.747|VBISARClu228887_0747|']

feature
latitude                                                                latitude
longitude                                                              longitude
depth_sampled                                                      depth_sampled
chl_satellite                                                      chl_satellite
sst_satellite                                                      sst_satellite
par_satellite                                                      par_satellite
pic_satellite                                                      pic_satellite
poc_satellite                                                      poc_satellite
npp_satellite                                                      npp_satellite
chla_monthly_historical                                  chla_monthly_historical
cloudfraction_monthly_historical                cloudfraction_monthly_historical
daylength_monthly_historical                        daylength_monthly_historical
dustflux_monthly_his

feature
latitude                                  0.000000
longitude                                 0.072304
depth_sampled                             0.114609
chl_satellite                             0.000000
sst_satellite                             0.000000
par_satellite                             0.000000
pic_satellite                            -0.693398
poc_satellite                             0.416967
npp_satellite                             0.000000
chla_monthly_historical                   0.000000
cloudfraction_monthly_historical          0.921967
daylength_monthly_historical              0.430903
dustflux_monthly_historical               0.000000
solarinsolation_monthly_historical        0.989800
pycnoclinedepth_monthly_historical        0.352641
thermoclinedepth_monthly_historical      -0.019875
nitrate_monthly_historical               -0.401475
npratio_monthly_historical                0.000000
oxygendissolved_monthly_historical        0.000000
oxygensaturation_monthl

### 1 - fn to get 2160x4320 lat/lon matrices for each feat

In [147]:
def get_feat_matrices(depth_sampled=-0.556543241, satellite_month=0, monthly_month=0, year_sampled=0, month_sampled=0):
    #this also requires contemp, monthly, annual, 
    #satellite_feats, monthly_feats, annual_feats, year_sampled_feats, month_sampled_feats
    
    feat_matrices = []
    #extract 2160x4320 matrix for each feature in same order as weights, append to list

    ###general descriptors
    print('getting general feats')
    lats = contemp['lat'][:].reshape(2160,1)
    lat_matrix = np.repeat(lats, 4320, axis=1)
    #normalize, subtract mean and divide by std
    lat_matrix = (lat_matrix - np.nanmean(lat_matrix))/float(np.nanstd(lat_matrix))

    lons = contemp['lon'][:].reshape(1,4320)
    lon_matrix = np.repeat(lons, 2160, axis=0)
    lon_matrix = (lon_matrix - np.nanmean(lon_matrix))/float(np.nanstd(lon_matrix))
    
    #depth_sampled is all surface for now, stealing that value from TARA env_remote_data_preprocessed
    depth_matrix = np.broadcast_to(depth_sampled, (2160,4320))

    feat_matrices.extend([lat_matrix, lon_matrix, depth_matrix])

    ###satellite data
    print('getting satellite feats')
    #satellite_month 0 for jan, 1 for apr, 2 for july, 3 for oct
    for sat in satellite_feats:
        mat = contemp[sat][:,:,satellite_month]
        #fill if it's a masked array
        if np.ma.is_masked(mat):
            mat = mat.filled(fill_value=np.nan)
        #normalize, subtract mean and divide by std
        mat = (mat - np.nanmean(mat))/float(np.nanstd(mat))
        #add to feat_matrices
        feat_matrices.extend([mat])

    ###monthly data
    print('getting monthly historical feats')
    #monthly_month 0 for jan, 3 for apr, 6 for july, 9 for oct
    for mo in monthly_feats:
        mat = monthly[mo][:,:,monthly_month]
        #normalize, subtract mean and divide by std
        mat = (mat - np.nanmean(mat))/float(np.nanstd(mat))
        #add to feat_matrices
        feat_matrices.extend([mat])

    ###annual data
    print('getting annual historical feats')
    for ann in annual_feats:
        mat = annual[ann][:,:]
        #normalize, subtract mean and divide by std
        mat = (mat - np.nanmean(mat))/float(np.nanstd(mat))
        #add to feat_matrices
        feat_matrices.extend([mat])

    ###year_sampled_feats
    print('getting year_sampled feats')
    #year_sampled 0 for 2009, 1 for 2010, 2 for 2011, 3 for 2012
    for yr_ix, yr in enumerate(year_sampled_feats):
        if yr_ix==year_sampled:
            mat = np.broadcast_to(1, shape=(2160,4320))
        else:
            mat = np.broadcast_to(0, shape=(2160,4320))
        #add to feat_matrices
        feat_matrices.extend([mat])

    ###month_sampled_feats
    print('getting month_sampled feats')
    #month_sampled 0 for jan, 1 for feb... 11 for dec
    for mo_ix, mo in enumerate(month_sampled_feats):
        if mo_ix==month_sampled:
            mat = np.broadcast_to(1, shape=(2160,4320))
        else:
            mat = np.broadcast_to(0, shape=(2160,4320))
        #add to feat_matrices
        feat_matrices.extend([mat])

    ###intercept
    print('adding intercept')
    int_mat = np.broadcast_to(1, shape=(2160,4320))
    feat_matrices.extend([int_mat])
    
    print('DONE!')
    return np.array(feat_matrices)

In [22]:
#test the above
feat_matrix = get_feat_matrices(depth_sampled=-0.556543241, satellite_month=0, monthly_month=0, year_sampled=0, month_sampled=0)

getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!


In [23]:
#should all have mean of ~0 and stdev of ~1 (except one-hot and depth_sampled), and be 2160x4320; they do
for arr in feat_matrix:
    print(arr.shape)
    print(np.nanmean(arr),np.nanstd(arr))

(2160, 4320)
-2.52430297363e-16 1.0
(2160, 4320)
-5.42166525057e-18 1.0
(2160, 4320)
-0.556543241 1.13242748512e-14
(2160, 4320)
2.78526346452e-16 1.0
(2160, 4320)
-3.65155102215e-16 1.0
(2160, 4320)
-1.31799362349e-15 1.0
(2160, 4320)
-5.64029178676e-17 1.0
(2160, 4320)
-1.10574591432e-15 1.0
(2160, 4320)
4.38499512833e-16 1.0
(2160, 4320)
3.66030842451e-16 1.0
(2160, 4320)
1.96132889259e-15 1.0
(2160, 4320)
-3.73418717073e-15 1.0
(2160, 4320)
2.93916697108e-16 1.0
(2160, 4320)
2.24670152928e-15 1.0
(2160, 4320)
9.48479281547e-16 1.0
(2160, 4320)
-4.85386797133e-15 1.0
(2160, 4320)
-5.23376091827e-16 1.0
(2160, 4320)
2.28197928597e-16 1.0
(2160, 4320)
1.05592367225e-15 1.0
(2160, 4320)
-2.02336897498e-14 1.0
(2160, 4320)
-2.18587144798e-16 1.0
(2160, 4320)
3.48414661103e-16 1.0
(2160, 4320)
-8.00670954163e-15 1.0
(2160, 4320)
-5.72588771035e-16 1.0
(2160, 4320)
-1.22306895658e-16 1.0
(2160, 4320)
1.91113244763e-16 1.0
(2160, 4320)
-1.55064162116e-16 1.0
(2160, 4320)
-6.27941086846e-16

### 2 - fn to multiply vector of weights by feat matrix ==> score matrix

In [34]:
def get_score_matrix(feat_matrix, weights):
    #broadcast weights to same size each feat's matrix so can multiply them together
    broadcast_weights = np.array([np.broadcast_to(w, shape=(2160,4320)) for w in weights])
    assert broadcast_weights.shape==feat_matrix.shape, "feat and weight matrices are not same shape"

    #multiply each value in each feat matrix by its coefficient weight 
    weighted_matrix = feat_matrix*broadcast_weights
    assert weighted_matrix.shape==feat_matrix.shape, "weighted feat and feat matrices are not same shape"
    
    #sum all 67 matrices to get score matrix
    scores = weighted_matrix.sum(axis=0)
    
    return scores

In [35]:
testgene_scores = get_score_matrix(feat_matrix, weights=balanced['fig|1123865.4.peg.747|VBISARClu228887_0747|'])

In [45]:
#dims should be 2160x4320; they are, and the range of scores is reasonable
print(testgene_scores.shape)
print(np.nanmean(testgene_scores))
print(np.nanmax(testgene_scores))
print(np.nanmin(testgene_scores))

(2160, 4320)
0.943763561192
8.8272197634
-19.4043945521


### 3 - fn to apply sigmoid fn to convert scores to probabilities

the sigmoid function is $sigmoid( w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}$

In [135]:
def get_sigmoid_matrix(score_matrix):
    #apply the sigmoid function to any scalar or array of any size
    sigmoid = 1.0/(1+np.exp(-score_matrix))
    return sigmoid

In [136]:
testsig_matrix = get_sigmoid_matrix(testgene_scores)

In [137]:
#shape is correct, mean and std look reasonable and is within [0,1]; as expected
print(testsig_matrix.shape)
print(np.nanmean(testsig_matrix), np.nanstd(testsig_matrix)) 
print(np.nanmax(testsig_matrix), np.nanmin(testsig_matrix))

(2160, 4320)
0.677460501586 0.240832187887
0.99985333602 3.73919849042e-09


### 4 - fn to predict presence/absence

In [134]:
def predict_pres_abs(sigmoid_matrix):
    prediction_matrix = sigmoid_matrix.copy()
    prediction_matrix[prediction_matrix>=0.5] = 1
    prediction_matrix[prediction_matrix<0.5] = 0
    return prediction_matrix

In [138]:
testpredictions = predict_pres_abs(testsig_matrix)

/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


In [139]:
#this looks about right
testpredictions[260:350,:]

array([[ nan,  nan,  nan, ...,   0.,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,   0.,   0.],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [  1.,   1.,   1., ...,   1.,   1.,   1.],
       [  1.,   1.,   1., ...,   1.,   1.,   1.],
       [ nan,   1.,   1., ...,   1.,   1.,   1.]])

## put it all together

In [141]:
def create_prediction_matrix(depth_sampled=-0.556543241, satellite_month=0, monthly_month=0, 
                                year_sampled=0, month_sampled=0, gene='fig|1123865.4.peg.747|VBISARClu228887_0747|'):
    print("creating feature matrices...")
    feat_matrix = get_feat_matrices(depth_sampled=depth_sampled, satellite_month=satellite_month, monthly_month=monthly_month, 
                                    year_sampled=year_sampled, month_sampled=month_sampled)
    print("scoring the matrix...")
    testgene_scores = get_score_matrix(feat_matrix, weights=balanced[gene])
    print("squishing through the sigmoid...")
    testsig_matrix = get_sigmoid_matrix(testgene_scores)
    print("making predictions...")
    testpredictions = predict_pres_abs(testsig_matrix)
    print("Eureka!")
    return testsig_matrix, testpredictions

In [119]:
testsig, testpredictions = create_prediction_matrix(depth_sampled=-0.556543241, satellite_month=0, monthly_month=0, 
                                year_sampled=0, month_sampled=0, gene='fig|1123865.4.peg.747|VBISARClu228887_0747|')

testpredictions[260:350,:]

creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...
Eureka!


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


array([[ nan,  nan,  nan, ...,   0.,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,   0.,   0.],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [  1.,   1.,   1., ...,   1.,   1.,   1.],
       [  1.,   1.,   1., ...,   1.,   1.,   1.],
       [ nan,   1.,   1., ...,   1.,   1.,   1.]])

## pick some genes to make maps of

In [275]:
annotations = pd.read_csv('Annotations_SAR86Agenes.csv')
set(annotations['ec_description'])

{nan,
 'D-alanine--D-alanine ligase',
 'Glutathione peroxidase',
 'UDP-3-O-acyl-N-acetylglucosamine deacetylase',
 'Lactaldehyde dehydrogenase',
 "3'(2'),5'-bisphosphate nucleotidase",
 '4-hydroxy-tetrahydrodipicolinate synthase',
 "Tetraacyldisaccharide 4'-kinase",
 'Thiolester hydrolases',
 'Exopolyphosphatase',
 'Asparaginyl-tRNA synthase (glutamine-hydrolyzing)',
 '[Acyl-carrier-protein] S-malonyltransferase',
 'Adenosylhomocysteinase',
 'DNA-directed RNA polymerase',
 'Acetyl-CoA C-acetyltransferase',
 'Long-chain-fatty-acid--CoA ligase',
 'Dihydroorotase',
 'dUTP diphosphatase',
 'Choline dehydrogenase',
 'Phosphatidate cytidylyltransferase',
 'Hypoxanthine phosphoribosyltransferase',
 'Aspartate carbamoyltransferase',
 'Valine--tRNA ligase',
 'Acrylyl-CoA reductase (NADPH)',
 'Glucose-6-phosphate isomerase',
 'Nucleotidyltransferases',
 '5-(carboxyamino)imidazole ribonucleotide mutase',
 'Cysteine--tRNA ligase',
 "4-(cytidine 5'-diphospho)-2-C-methyl-D-erythritol kinase",
 'Glut

In [276]:
annotations

,gene,pathway_name,ec_description,ec_number,ERR598993,ERR598969,ERR599106,ERR599097,ERR599041,ERR599116,...,ERR599029,ERR599100,ERR599136,ERR598983,ERR599123,ERR598963,ERR598996,ERR599170,ERR598986,ERR598976
0,fig|1123866.3.peg.1001|VBISARClu227658_1001|,Aminoacyl-tRNA biosynthesis,Arginine--tRNA ligase,6.1.1.19,18.630983,4.517186,0.541001,5.706419,2.529651,1.652240,...,0.108282,0.118920,0.111042,0.062424,0.208940,0.054687,0.225927,0.232889,0.423729,0.933389
1,fig|1123866.3.peg.1002|VBISARClu227658_1002|,NaN,NaN,NaN,32.943422,12.027980,1.654930,14.017471,7.831640,4.486637,...,0.298937,0.330885,0.283766,0.218634,0.628148,0.184794,0.523628,0.624029,2.007776,3.659077
2,fig|1123866.3.peg.1003|VBISARClu227658_1003|,NaN,NaN,NaN,31.384264,12.401618,1.583563,12.566618,8.096534,4.588577,...,0.263364,0.346458,0.320385,0.334770,0.592168,0.196740,0.432384,0.605323,2.599972,3.595294
3,fig|1123866.3.peg.1004|VBISARClu227658_1004|,Arginine and proline metabolism,Pyrroline-5-carboxylate reductase,1.5.1.2,28.749913,12.391284,1.322200,12.279412,7.863651,4.257078,...,0.282949,0.333723,0.307213,0.149745,0.606882,0.169929,0.527774,0.609747,1.839967,2.023212
4,fig|1123866.3.peg.1005|VBISARClu227658_1005|,NaN,NaN,NaN,29.610106,11.547585,1.558840,14.839634,7.310972,4.497894,...,0.302254,0.307328,0.321399,0.150639,0.697206,0.159223,0.510245,0.625779,1.667299,3.254818
5,fig|1123866.3.peg.1006|VBISARClu227658_1006|,Cysteine and methionine metabolism,Homoserine O-acetyltransferase,2.3.1.31,16.312660,6.371167,0.909995,8.883951,3.608012,2.305917,...,0.128784,0.189423,0.197954,0.117840,0.326685,0.109624,0.289083,0.332766,1.105119,1.865866
6,fig|1123866.3.peg.1006|VBISARClu227658_1006|,Sulfur metabolism,Homoserine O-acetyltransferase,2.3.1.31,16.312660,6.371167,0.909995,8.883951,3.608012,2.305917,...,0.128784,0.189423,0.197954,0.117840,0.326685,0.109624,0.289083,0.332766,1.105119,1.865866
7,fig|1123866.3.peg.1007|VBISARClu227658_1007|,NaN,NaN,NaN,13.783666,4.572427,0.680706,7.775060,3.521755,2.216143,...,0.118638,0.126745,0.140236,0.107256,0.172216,0.095049,0.308056,0.366820,0.505804,1.154296
8,fig|1123866.3.peg.1009|VBISARClu227658_1009|,NaN,NaN,NaN,26.718764,6.003043,0.790633,5.684062,3.879401,2.503730,...,0.158910,0.177702,0.172567,0.149145,0.354641,0.071152,0.223860,0.324419,0.494562,1.065303
9,fig|1123866.3.peg.1010|VBISARClu227658_1010|,NaN,NaN,NaN,39.369734,12.440709,1.516373,11.751840,6.271282,3.715529,...,0.262804,0.295439,0.329783,0.198783,0.591386,0.163785,0.464944,0.502069,1.618211,4.357994


In [277]:
set(annotations['pathway_name'])

{nan,
 '1,4-Dichlorobenzene degradation',
 'Pyrimidine metabolism',
 'Zeatin biosynthesis',
 'One carbon pool by folate',
 'Inositol phosphate metabolism',
 'Porphyrin and chlorophyll metabolism',
 'Valine, leucine and isoleucine biosynthesis',
 'Lysine degradation',
 'gamma-Hexachlorocyclohexane degradation',
 'Ethylbenzene degradation',
 'Pyruvate metabolism',
 'Nicotinate and nicotinamide metabolism',
 'Naphthalene and anthracene degradation',
 'Metabolism of xenobiotics by cytochrome P450',
 'Oxidative phosphorylation',
 'Ascorbate and aldarate metabolism',
 'Glutathione metabolism',
 'Phenylalanine metabolism',
 'Thiamine metabolism',
 'Pentose and glucuronate interconversions',
 'Pentose phosphate pathway',
 'Arachidonic acid metabolism',
 'Folate biosynthesis',
 'Sulfur metabolism',
 'Caprolactam degradation',
 '2,4-Dichlorobenzoate degradation',
 'Butanoate metabolism',
 'Reductive carboxylate cycle (CO2 fixation)',
 'Ubiquinone and other terpenoid-quinone biosynthesis',
 'Prop

In [80]:
moreannotations = pd.read_csv('../annotation_SAR86.tab', sep='\t')
moreannotations.head()

,orf_id,orf_contam_type,kegg_hit,kegg_desc,kegg_pathway,KO,KO_desc,KO_pathway,EC,uniprot,...,best_hit_group,PFams,PFams_desc,TIGRFams,TIGRFams_desc,transporter_family,transporter_substrate,transporter_evidence,transmembrane_domains,Unnamed: 33
0,fig|1007118.3.peg.1000|VBIGamPro251610_1000|,NaN,hch:HCH_00048,coxA; cytochrome-c oxidase subunit I (EC:1.9.3...,hch00190||hch01100,K02274,cytochrome c oxidase subunit I [EC:1.9.3.1],Oxidative phosphorylation,1.9.3.1,Q2SQV4,...,NaN,PF00115||_GAP_,Cytochrome C and Quinol oxidase polypeptide I|...,TIGR02891,"cytochrome c oxidase, subunit I",NaN,NaN,NaN,12,NaN
1,fig|1007118.3.peg.1001|VBIGamPro251610_1001|,NaN,pae:PA0105 || pau:PA14_01290 || pag:PLES_01061,"coxB; cytochrome c oxidase, subunit II; K02275...",pae00190||pae01100,K02275,cytochrome c oxidase subunit II [EC:1.9.3.1],Oxidative phosphorylation,1.9.3.1,Q9I727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,fig|1007118.3.peg.1002|VBIGamPro251610_1002|,NaN,chu:CHU_2889,rgpA; A-glycosyltransferase,NaN,NaN,NaN,NaN,NaN,Q11R28,...,NaN,PF09314||_GAP_,Domain of unknown function (DUF1972)||_GAP_,NaN,NaN,NaN,NaN,NaN,1,NaN
3,fig|1007118.3.peg.1003|VBIGamPro251610_1003|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,fig|1007118.3.peg.1004|VBIGamPro251610_1004|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,TIGR04396||_GAP_,TIGR04396||_GAP_,NaN,NaN,NaN,0,NaN


In [83]:
moreannotations.columns

Index(['orf_id', 'orf_contam_type', 'kegg_hit', 'kegg_desc', 'kegg_pathway',
       'KO', 'KO_desc', 'KO_pathway', 'EC', 'uniprot', 'KOG_id', 'KOG_desc',
       'KOG_class', 'KOG_group', 'organelle', 'organelle_id',
       'organelle_species', 'organelle_e-value', 'best_hit',
       'best_hit_percent_identity', 'best_hit_annotation',
       'best_hit_GOS_core_cluster', 'best_hit_species', 'best_hit_taxon_id',
       'best_hit_group', 'PFams', 'PFams_desc', 'TIGRFams', 'TIGRFams_desc',
       'transporter_family', 'transporter_substrate', 'transporter_evidence',
       'transmembrane_domains', 'Unnamed: 33'],
      dtype='object')

In [88]:
from collections import Counter
Counter(moreannotations[moreannotations['best_hit_percent_identity']>95]['best_hit_annotation']).most_common()

[('conserved hypothetical protein', 639),
 ('hypothetical protein', 320),
 ('putative membrane protein', 54),
 ('TonB-dependent receptor', 44),
 ('KR domain protein', 36),
 ('putative lipoprotein', 34),
 ('30S ribosomal protein S11', 30),
 ('DNA-directed RNA polymerase, alpha subunit', 28),
 ('acyl-CoA dehydrogenase domain protein', 28),
 ('putative TonB-dependent receptor', 27),
 ('RecA protein', 25),
 ('transcription termination factor Rho', 25),
 ('cytochrome c oxidase, subunit I', 24),
 ('RNA polymerase sigma factor RpoD', 23),
 ('biopolymer transport protein exbD2', 23),
 ('preprotein translocase, SecY subunit', 22),
 ('transporter, major facilitator family', 21),
 ('ribosomal protein S5', 21),
 ('MFS transporter', 20),
 ('acyl-CoA dehydrogenase, C-terminal domain protein', 20),
 ('ATPase', 20),
 ('phospholipid/glycerol acyltransferase', 19),
 ('IMG gene_oid=2236280709', 19),
 ('V-type H(+)-translocating pyrophosphatase', 19),
 ('ribosomal protein S4', 18),
 ('ribosomal protein L2

In [89]:
confident = moreannotations[moreannotations['best_hit_percent_identity']>95]
print(len(confident))
confident.tail()

12579


,orf_id,orf_contam_type,kegg_hit,kegg_desc,kegg_pathway,KO,KO_desc,KO_pathway,EC,uniprot,...,best_hit_group,PFams,PFams_desc,TIGRFams,TIGRFams_desc,transporter_family,transporter_substrate,transporter_evidence,transmembrane_domains,Unnamed: 33
62942,scf7180009409913_10713_10375_10,NaN,apb:SAR116_1570,catalase (EC:1.11.1.6); K03782 catalase/peroxi...,apb00360||apb01120||apb01110||apb00680||apb011...,K03782,catalase-peroxidase [EC:1.11.1.21],Phenylalanine metabolism || Tryptophan metabol...,1.11.1.6||1.11.1.7,D5BU66,...,NaN,PF00141,Peroxidase,NaN,NaN,NaN,NaN,NaN,0,NaN
62946,scf7180009409913_12968_12501_12,NaN,pjd:Pjdr2_1725,catalase/peroxidase HPI; K03782 catalase/perox...,pjd01110||pjd00360||pjd00380||pjd00680||pjd011...,K03782,catalase-peroxidase [EC:1.11.1.21],Phenylalanine metabolism || Tryptophan metabol...,1.11.1.6||1.11.1.7,C6CXH4,...,NaN,_GAP_||PF00141,_GAP_||Peroxidase,NaN,NaN,NaN,NaN,NaN,0,NaN
62947,scf7180009409913_13107_13529_13,NaN,cps:CPS_1346,LysR family transcriptional regulator; K04761 ...,NaN,K04761,"LysR family transcriptional regulator, hydroge...",NaN,NaN,Q486C6,...,NaN,PF00126||PF03466,"Bacterial regulatory helix-turn-helix protein,...",NaN,NaN,NaN,NaN,NaN,0,NaN
62960,scf7180009409913_26044_25184_25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PF02551||PF13622||PF02551,Acyl-CoA thioesterase||PF13622||Acyl-CoA thioe...,NaN,NaN,NaN,NaN,NaN,0,NaN
62963,scf7180009409913_28976_30082_28,NaN,abo:ABO_1416,hypothetical protein,NaN,NaN,NaN,NaN,NaN,Q0VPN4,...,NaN,_GAP_||PF11583||_GAP_,_GAP_||P-aminobenzoate N-oxygenase AurF||_GAP_,NaN,NaN,NaN,NaN,NaN,0,NaN


In [90]:
Counter(confident['KO_pathway']).most_common()

[(nan, 8635),
 ('Ribosome', 709),
 ('Aminoacyl-tRNA biosynthesis', 108),
 ('Oxidative phosphorylation || Photosynthesis', 90),
 ('Protein export || Bacterial secretion system', 90),
 ('Phenylalanine, tyrosine and tryptophan biosynthesis || Biosynthesis of amino acids',
  84),
 ('ABC transporters', 83),
 ('Porphyrin and chlorophyll metabolism', 77),
 ('Purine metabolism', 63),
 ('Pyrimidine metabolism', 61),
 ('Lipopolysaccharide biosynthesis', 59),
 ('Purine metabolism || Pyrimidine metabolism || RNA polymerase', 57),
 ('Cell cycle - Caulobacter', 53),
 ('Nicotinate and nicotinamide metabolism', 50),
 ('Terpenoid backbone biosynthesis', 50),
 ('RNA degradation', 47),
 ('Base excision repair', 45),
 ('Cysteine and methionine metabolism', 45),
 ('Oxidative phosphorylation', 41),
 ('Bacterial secretion system', 40),
 ('Two-component system', 40),
 ('Purine metabolism || Pyrimidine metabolism || DNA replication || Mismatch repair || Homologous recombination',
  38),
 ('Fatty acid degradati

In [91]:
Counter(confident['best_hit_annotation']).most_common()

[('conserved hypothetical protein', 639),
 ('hypothetical protein', 320),
 ('putative membrane protein', 54),
 ('TonB-dependent receptor', 44),
 ('KR domain protein', 36),
 ('putative lipoprotein', 34),
 ('30S ribosomal protein S11', 30),
 ('DNA-directed RNA polymerase, alpha subunit', 28),
 ('acyl-CoA dehydrogenase domain protein', 28),
 ('putative TonB-dependent receptor', 27),
 ('RecA protein', 25),
 ('transcription termination factor Rho', 25),
 ('cytochrome c oxidase, subunit I', 24),
 ('RNA polymerase sigma factor RpoD', 23),
 ('biopolymer transport protein exbD2', 23),
 ('preprotein translocase, SecY subunit', 22),
 ('transporter, major facilitator family', 21),
 ('ribosomal protein S5', 21),
 ('MFS transporter', 20),
 ('acyl-CoA dehydrogenase, C-terminal domain protein', 20),
 ('ATPase', 20),
 ('phospholipid/glycerol acyltransferase', 19),
 ('IMG gene_oid=2236280709', 19),
 ('V-type H(+)-translocating pyrophosphatase', 19),
 ('ribosomal protein S4', 18),
 ('ribosomal protein L2

In [326]:
confident[confident['best_hit_annotation']=='TonB-dependent receptor']

,orf_id,orf_contam_type,kegg_hit,kegg_desc,kegg_pathway,KO,KO_desc,KO_pathway,EC,uniprot,...,best_hit_group,PFams,PFams_desc,TIGRFams,TIGRFams_desc,transporter_family,transporter_substrate,transporter_evidence,transmembrane_domains,Unnamed: 33
6877,fig|1123866.3.peg.1214|VBISARClu227658_1214|,NaN,son:SO_2715,TonB-dependent receptor,NaN,NaN,NaN,NaN,NaN,Q8EDM8,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,1,NaN
7151,fig|1123866.3.peg.307|VBISARClu227658_0307|,NaN,sdn:Sden_2610,TonB-dependent receptor,NaN,NaN,NaN,NaN,NaN,Q12KY7,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,TIGR01782,TonB-dependent receptor,NaN,NaN,NaN,0,NaN
7247,fig|1123866.3.peg.394|VBISARClu227658_0394|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
7260,fig|1123866.3.peg.405|VBISARClu227658_0405|,NaN,ttu:TERTU_3776,TonB-dependent receptor,NaN,NaN,NaN,NaN,NaN,C5BST4,...,NaN,_GAP_||PF07715||_GAP_,_GAP_||TonB-dependent Receptor Plug Domain||_GAP_,NaN,NaN,NaN,NaN,NaN,0,NaN
7455,fig|1123866.3.peg.581|VBISARClu227658_0581|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593||_GAP_,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
7462,fig|1123866.3.peg.588|VBISARClu227658_0588|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
7700,fig|1123866.3.peg.801|VBISARClu227658_0801|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
8034,fig|1123867.3.peg.1102|VBISARClu228688_1102|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,1,NaN
8145,fig|1123867.3.peg.1202|VBISARClu228688_1202|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
8307,fig|1123867.3.peg.1349|VBISARClu228688_1349|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF00593,_GAP_||TonB dependent receptor,NaN,NaN,NaN,NaN,NaN,0,NaN


In [115]:
#to start, let's use the proteins with the annotation 'TonB-dependent receptor'] that have a transmembrane domain (6 of these)
tonb = confident[confident['best_hit_annotation']=='TonB-dependent receptor']
tonb_genes = []
#check if any of these genes in balanced genes
for gene in tonb['orf_id']:
    if gene in balanced.columns:
        tonb_genes.append(gene)
tonb = tonb[tonb['orf_id'].isin(tonb_genes)]
tonb

,orf_id,orf_contam_type,kegg_hit,kegg_desc,kegg_pathway,KO,KO_desc,KO_pathway,EC,uniprot,...,best_hit_group,PFams,PFams_desc,TIGRFams,TIGRFams_desc,transporter_family,transporter_substrate,transporter_evidence,transmembrane_domains,Unnamed: 33
7260,fig|1123866.3.peg.405|VBISARClu227658_0405|,NaN,ttu:TERTU_3776,TonB-dependent receptor,NaN,NaN,NaN,NaN,NaN,C5BST4,...,NaN,_GAP_||PF07715||_GAP_,_GAP_||TonB-dependent Receptor Plug Domain||_GAP_,NaN,NaN,NaN,NaN,NaN,0,NaN
7455,fig|1123866.3.peg.581|VBISARClu227658_0581|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593||_GAP_,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
7700,fig|1123866.3.peg.801|VBISARClu227658_0801|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
10769,fig|1208365.4.peg.742|B273_1132|VBISARClu25576...,NaN,xcc:XCC2658 || xcb:XC_1459,phuR; outer membrane hemin receptor; K02014 ir...,NaN,K02014,iron complex outermembrane recepter protein,NaN,NaN,Q8P7F3,...,NaN,_GAP_||PF07715||_GAP_||PF14905||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
48190,scf7180009407180_47146_43970_43,NaN,sdn:Sden_2610,TonB-dependent receptor,NaN,NaN,NaN,NaN,NaN,Q12KY7,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,TIGR01782,TonB-dependent receptor,NaN,NaN,NaN,0,NaN
54070,scf7180009408934_79519_81909_77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN
57804,scf7180009409220_182471_185401_181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,_GAP_||PF07715||_GAP_||PF00593,_GAP_||TonB-dependent Receptor Plug Domain||_G...,NaN,NaN,NaN,NaN,NaN,0,NaN


## make some maps - or rather, save an .nc file and make some maps in panoply 

In [78]:
years = [2009, 2010, 2011, 2012]
months = [1,2,3,4,5,6,7,8,9,10,11,12]

### create a 2009Jan and 2009July .nc file with just lat/lon

In [142]:
jan2009 = nc.Dataset("Jan2009_projections.nc", "r+", format="NETCDF4")

In [103]:
#add dimensions - 
#contemporary dims should be lat, lon, and time which will be unlimited (but for now 4 layers with quarterly months (Jan 2009, Apr 2009, July 2009, Oct 2009))
lat = jan2009.createDimension("lat", 2160)
lon = jan2009.createDimension("lon", 4320)

#create lat and lon and time variables as 64-bit floats and ints
latitudes = jan2009.createVariable("lat","f8",("lat",))
longitudes = jan2009.createVariable("lon","f8",("lon",))

#assign lat/lon values at 9km res to the lat/lon variables (same as contemp)
jan2009['lat'][:] = contemp['lat'][:]
jan2009['lon'][:] = contemp['lon'][:]

In [143]:
for tonb_gene in tonb['orf_id']:
    print('---processing gene ', tonb_gene, '---')
    sigmoids, tonb_predictions = create_prediction_matrix(depth_sampled=-0.556543241, satellite_month=0, monthly_month=0, 
                                year_sampled=0, month_sampled=0, gene=tonb_gene)
    #create a variable in our nc file for the sig and pred if it doesn't exist, else just add data
    if 'sig_'+str(tonb_gene) in jan2009.variables:
        jan2009['sig_'+str(tonb_gene)][:] = sigmoids
    else:
        jan2009.createVariable('sig_'+str(tonb_gene), "f8", ("lat", "lon"))
        jan2009['sig_'+str(tonb_gene)][:] = sigmoids
        
    if 'pred_'+str(tonb_gene) in jan2009.variables:
        jan2009['pred_'+str(tonb_gene)][:] = tonb_predictions
    else:
        jan2009.createVariable('pred_'+str(tonb_gene), "f8", ("lat","lon"))
        jan2009['pred_'+str(tonb_gene)][:] = tonb_predictions


---processing gene  fig|1123866.3.peg.405|VBISARClu227658_0405| ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


Eureka!
---processing gene  fig|1123866.3.peg.581|VBISARClu227658_0581| ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


Eureka!
---processing gene  fig|1123866.3.peg.801|VBISARClu227658_0801| ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...

...

Eureka!
---processing gene  scf7180009408934_79519_81909_77 ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


Eureka!
---processing gene  scf7180009409220_182471_185401_181 ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


Eureka!


#### close to save content

In [144]:
jan2009.close()

In [145]:
july2009 = nc.Dataset("July2009_projections.nc", "r+", format="NETCDF4")

In [ ]:
#add dimensions - 
#contemporary dims should be lat, lon, and time which will be unlimited (but for now 4 layers with quarterly months (Jan 2009, Apr 2009, July 2009, Oct 2009))
lat = july2009.createDimension("lat", 2160)
lon = july2009.createDimension("lon", 4320)

#create lat and lon and time variables as 64-bit floats and ints
latitudes = july2009.createVariable("lat","f8",("lat",))
longitudes = july2009.createVariable("lon","f8",("lon",))

#assign lat/lon values at 9km res to the lat/lon variables (same as contemp)
july2009['lat'][:] = contemp['lat'][:]
july2009['lon'][:] = contemp['lon'][:]

In [150]:
for tonb_gene in tonb['orf_id']:
    print('---processing gene ', tonb_gene, '---')
    sigmoids, tonb_predictions = create_prediction_matrix(depth_sampled=-0.556543241, satellite_month=2, 
                                monthly_month=6, year_sampled=0, month_sampled=6, gene=tonb_gene)
    #create a variable in our nc file for the sig and pred if it doesn't exist, else just add data
    if 'sig_'+str(tonb_gene) in july2009.variables:
        july2009['sig_'+str(tonb_gene)][:] = sigmoids
    else:
        july2009.createVariable('sig_'+str(tonb_gene), "f8", ("lat", "lon"))
        july2009['sig_'+str(tonb_gene)][:] = sigmoids
        
    if 'pred_'+str(tonb_gene) in july2009.variables:
        july2009['pred_'+str(tonb_gene)][:] = tonb_predictions
    else:
        july2009.createVariable('pred_'+str(tonb_gene), "f8", ("lat","lon"))
        july2009['pred_'+str(tonb_gene)][:] = tonb_predictions


---processing gene  fig|1123866.3.peg.405|VBISARClu227658_0405| ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...
Eureka!


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


---processing gene  fig|1123866.3.peg.581|VBISARClu227658_0581| ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


Eureka!
---processing gene  fig|1123866.3.peg.801|VBISARClu227658_0801| ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...
Eureka!


...

/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


---processing gene  scf7180009408934_79519_81909_77 ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


Eureka!
---processing gene  scf7180009409220_182471_185401_181 ---
creating feature matrices...
getting general feats
getting satellite feats
getting monthly historical feats
getting annual historical feats
getting year_sampled feats
getting month_sampled feats
adding intercept
DONE!
scoring the matrix...
squishing through the sigmoid...
making predictions...
Eureka!


/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater_equal
  app.launch_new_instance()
/Users/Adrienne/anaconda/envs/nb/lib/python3.4/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


In [151]:
july2009.close()

### wow a couple of  these genes looks super different

why?


In [152]:
balanced[['fig|1123866.3.peg.405|VBISARClu227658_0405|', 'fig|1208365.4.peg.742|B273_1132|VBISARClu255765_0742|']]

,fig|1123866.3.peg.405|VBISARClu227658_0405|,fig|1208365.4.peg.742|B273_1132|VBISARClu255765_0742|
feature,,
latitude,0.000000,0.000000
longitude,0.000000,0.114294
depth_sampled,0.559541,0.000000
chl_satellite,0.000000,0.000000
sst_satellite,1.151270,0.000000
par_satellite,0.000000,0.000000
pic_satellite,0.000000,-0.631583
poc_satellite,-0.270345,0.946314
npp_satellite,0.000000,0.513865
